In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display='diagram')

In [2]:
path = './medical_data - medical_data.csv'
md_df = pd.read_csv(path)
md_df.info()
md_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               995 non-null    object 
 1   Lat                 1000 non-null   float64
 2   Lng                 1000 non-null   float64
 3   Area                995 non-null    object 
 4   Children            993 non-null    float64
 5   Age                 1000 non-null   int64  
 6   Income              1000 non-null   float64
 7   Marital             995 non-null    object 
 8   Gender              995 non-null    object 
 9   ReAdmis             1000 non-null   int64  
 10  VitD_levels         1000 non-null   float64
 11  Doc_visits          1000 non-null   int64  
 12  Full_meals_eaten    1000 non-null   int64  
 13  vitD_supp           1000 non-null   int64  
 14  Soft_drink          1000 non-null   int64  
 15  Initial_admin       995 non-null    object 
 16  HighBlo

,State,Lat,Lng,Area,Children,Age,Income,Marital,Gender,ReAdmis,...,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,Services,Initial_days,TotalCharge,Additional_charges
0,AL,34.34960,-86.72508,Suburban,1.0,53,86575.93,Divorced,Male,0,...,0.0,1.0,1.0,1.0,0,1,Blood Work,10.585770,3726.702860,17939.403420
1,FL,30.84513,-85.22907,Urban,3.0,51,46805.99,Married,Female,0,...,0.0,0.0,0.0,0.0,1,0,Intravenous,15.129562,4193.190458,17612.998120
2,SD,43.54321,-96.63772,Suburban,3.0,53,14370.14,Widowed,Female,0,...,0.0,0.0,0.0,0.0,0,0,Blood Work,4.772177,2434.234222,17505.192460
3,MN,43.89744,-93.51479,Suburban,0.0,78,39741.49,Married,Male,0,...,0.0,0.0,0.0,0.0,1,1,Blood Work,1.714879,2127.830423,12993.437350
4,VA,37.59894,-76.88958,Rural,1.0,22,1209.56,Widowed,Female,0,...,1.0,0.0,0.0,1.0,0,0,CT Scan,1.254807,2113.073274,3716.525786


In [3]:
print(md_df.isna().sum().sum(), 'missing values')

72 missing values


In [4]:
def check_df_null_percentage(df):
    missing_values_count= df.isna().sum()
    total_cells = np.product(df.shape)
    total_missing = missing_values_count.sum()
    total_rows = df.shape[0]
    percent_missing = (total_missing/total_cells) * 100
    # created percentage null checks for each row 
    for col in df:
        series = df[col]
        each_series_null_values = series.isna().sum()
        percentage_each_series = (each_series_null_values/total_rows) * 100
        percentage_each_series_df  = (each_series_null_values/total_cells) * 100
        text = (
            f'''Series column {col} has {each_series_null_values} missing values which is 
            {percentage_each_series.round(decimals=2)} % of row data or 
            {percentage_each_series_df.round(decimals=2)} of the whole dataset'''
        )
        print(text)
        # print(each_series_null_values, percentage_each_series, percentage_each_series_df)
    print(f"There are {total_missing} missing values which is {percent_missing.round(decimals=2)}% missing data in your dataset" )
    

In [5]:
check_df_null_percentage(md_df)

Series column State has 5 missing values which is 
            0.5 % of row data or 
            0.02 of the whole dataset
Series column Lat has 0 missing values which is 
            0.0 % of row data or 
            0.0 of the whole dataset
Series column Lng has 0 missing values which is 
            0.0 % of row data or 
            0.0 of the whole dataset
Series column Area has 5 missing values which is 
            0.5 % of row data or 
            0.02 of the whole dataset
Series column Children has 7 missing values which is 
            0.7 % of row data or 
            0.02 of the whole dataset
Series column Age has 0 missing values which is 
            0.0 % of row data or 
            0.0 of the whole dataset
Series column Income has 0 missing values which is 
            0.0 % of row data or 
            0.0 of the whole dataset
Series column Marital has 5 missing values which is 
            0.5 % of row data or 
            0.02 of the whole dataset
Series column Gender 

We could drop rows or columns before the validation split without leaking data.

However, this lesson is about imputation so we will impute the missing values instead.

Train/Test Split

Imputing missing values can leak information from the testing data into the training data, so we impute values AFTER we split the data.

First, we define our target, "Additional_charges" as y and our features (the rest of the columns) as X. Then we perform the train test split. 

In [6]:
target= 'Additional_charges'
X = md_df.drop(columns=[target])
y = md_df[target]
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Select Columns

We are going to separate our features into two types of columns based on the data type. One will be our numeric columns which will include both integers and floats. The other column with be the categorical columns that include our strings (objects). 

In [7]:
#instantiate the selectors to for numeric and categorical data types
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')
#select the numeric columns of each type
num_columns = num_selector(X_train)
cat_columns = cat_selector(X_train)
#check our lists
print('numeric columns are --->', num_columns)
print('\ncategorical columns are --->\n', cat_columns)

numeric columns are ---> ['Lat', 'Lng', 'Children', 'Age', 'Income', 'ReAdmis', 'VitD_levels', 'Doc_visits', 'Full_meals_eaten', 'vitD_supp', 'Soft_drink', 'HighBlood', 'Stroke', 'Overweight', 'Arthritis', 'Diabetes', 'Hyperlipidemia', 'BackPain', 'Anxiety', 'Allergic_rhinitis', 'Reflux_esophagitis', 'Asthma', 'Initial_days', 'TotalCharge']

categorical columns are --->
 ['State', 'Area', 'Marital', 'Gender', 'Initial_admin', 'Complication_risk', 'Services']


# SimpleImputer with ColumnTransformer

Let's recreate our original X_train with all of the missing values and see how ColumnTransformer, combined with SimpleImputer, can impute both the numeric columns with medians and the categorical columns with the most frequent value.
Create a New X_train With Missing Values

In [12]:
# Train Test Split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=42)
X_train2.isna().any()



State                  True
Lat                   False
Lng                   False
Area                   True
Children               True
Age                   False
Income                False
Marital                True
Gender                 True
ReAdmis               False
VitD_levels           False
Doc_visits            False
Full_meals_eaten      False
vitD_supp             False
Soft_drink            False
Initial_admin          True
HighBlood             False
Stroke                False
Complication_risk      True
Overweight            False
Arthritis              True
Diabetes               True
Hyperlipidemia         True
BackPain               True
Anxiety                True
Allergic_rhinitis      True
Reflux_esophagitis    False
Asthma                False
Services               True
Initial_days          False
TotalCharge           False
dtype: bool

# Re-Instantiate ColumnSelectors

In [13]:
#instantiate the selectors to for numeric and categorical data types
num_selector2 = make_column_selector(dtype_include='number')
cat_selector2 = make_column_selector(dtype_include='object')


# Instantiate Imputers

We will fill in missing data in numeric columns with the median of each column and missing data in categorical columns with the most frequent value. These are not the only options, but it's what we will do today.

In [14]:
#instantiate SimpleImputers with most_frequent and median strategies
freq_imputer = SimpleImputer(strategy='most_frequent')
median_imputer = SimpleImputer(strategy='median')


# Build the ColumnTransformer

As you recall from other lessons, make_column_transformer() takes tuples of the form (transformer, columns). ColumnSelectors can be used instead of lists of columns. Both are acceptable. We can set remainder='passthrough' if we are not applying transformers to all columns. We might do this if we imputed some columns by hand already. The default for ColumnTransformer is to drop any columns that are not specified in a tuple, while remainder = 'passthrough' retains the original values for any columns not specified in a tuple without any transformation.

In [15]:
# create tuples of (imputer, selector) for each datatype
num_tuple = (median_imputer, num_selector2)
cat_tuple = (freq_imputer, cat_selector2)
# instantiate ColumnTransformer
col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder='passthrough')
col_transformer


ColumnTransformer(remainder='passthrough',
                  transformers=[('simpleimputer-1',
                                 SimpleImputer(strategy='median'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x139969ed0>),
                                ('simpleimputer-2',
                                 SimpleImputer(strategy='most_frequent'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x13996ab60>)])

# Impute Missing Values With ColumnTransformer

In [16]:
# fit ColumnTransformer on the training data
col_transformer.fit(X_train2)
# transform both the training and testing data (this will output a NumPy array)
X_train_imputed = col_transformer.transform(X_train2)
X_test_imputed = col_transformer.transform(X_test2)
# change the result back to a dataframe
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_train_imputed.isna().any()



State                 False
Lat                   False
Lng                   False
Area                  False
Children              False
Age                   False
Income                False
Marital               False
Gender                False
ReAdmis               False
VitD_levels           False
Doc_visits            False
Full_meals_eaten      False
vitD_supp             False
Soft_drink            False
Initial_admin         False
HighBlood             False
Stroke                False
Complication_risk     False
Overweight            False
Arthritis             False
Diabetes              False
Hyperlipidemia        False
BackPain              False
Anxiety               False
Allergic_rhinitis     False
Reflux_esophagitis    False
Asthma                False
Services              False
Initial_days          False
TotalCharge           False
dtype: bool

You can see that our dataframe is now free of missing values. Using ColumnTransformer reduces the complexity of our code, reduces the chances for errors, and you will see later that we can use it with other tools to streamline the modeling process.
# Summary

SimpleImputer can impute missing values in many columns at once. It can also help avoid data leakage. If we use SimpleImputer and ColumnTransformer together, we can easily apply different imputation strategies to different columns simultaneously.

---------------------------------------------------------------------------------------------------------

Practicing One-Hot Encoding

In [19]:
X_train2

,State,Lat,Lng,Area,Children,Age,Income,Marital,Gender,ReAdmis,...,Diabetes,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,Services,Initial_days,TotalCharge
82,TN,36.16307,-86.66510,Urban,2.0,60,8459.99,Never Married,Female,0,...,0.0,1.0,1.0,0.0,0.0,0,0,Intravenous,6.714754,3097.337588
991,AL,34.96594,-87.12179,Urban,5.0,78,22669.31,Married,Male,0,...,0.0,1.0,1.0,0.0,0.0,1,0,Blood Work,5.694359,3073.408768
789,TN,36.24648,-83.51232,Urban,1.0,60,25536.25,Married,Nonbinary,0,...,0.0,0.0,1.0,0.0,0.0,1,0,Intravenous,7.336514,3199.418504
894,SD,45.42189,-97.91165,Rural,7.0,82,94863.57,Never Married,Male,0,...,1.0,0.0,1.0,0.0,0.0,0,0,Blood Work,13.172367,3693.118743
398,MI,42.33661,-83.28292,Suburban,0.0,37,30898.36,Widowed,Female,0,...,0.0,1.0,0.0,0.0,0.0,0,0,Blood Work,7.257809,2616.316061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,NY,42.05701,-77.43901,Suburban,1.0,32,4788.93,Widowed,Male,0,...,0.0,0.0,0.0,1.0,1.0,0,1,Intravenous,1.641752,2321.176987
270,IN,40.47773,-86.38658,Suburban,4.0,27,29461.62,Married,Male,0,...,0.0,0.0,0.0,0.0,1.0,0,0,Blood Work,15.908995,3633.310014
860,OH,40.56510,-81.07429,Urban,0.0,57,79094.04,Divorced,Male,0,...,0.0,0.0,0.0,0.0,0.0,1,1,Blood Work,20.220180,3644.114869
435,CA,33.97472,-118.35549,Suburban,0.0,56,25697.12,Separated,Male,0,...,0.0,0.0,0.0,0.0,0.0,0,0,Blood Work,11.798050,2894.343499


In [30]:
X_train2.isna().sum()

State                 2
Lat                   0
Lng                   0
Area                  3
Children              4
Age                   0
Income                0
Marital               3
Gender                4
ReAdmis               0
VitD_levels           0
Doc_visits            0
Full_meals_eaten      0
vitD_supp             0
Soft_drink            0
Initial_admin         3
HighBlood             0
Stroke                0
Complication_risk     1
Overweight            0
Arthritis             4
Diabetes              5
Hyperlipidemia        2
BackPain              5
Anxiety               1
Allergic_rhinitis     6
Reflux_esophagitis    0
Asthma                0
Services              3
Initial_days          0
TotalCharge           0
dtype: int64

In [18]:
from sklearn.preprocessing import OneHotEncoder

In [28]:
def check_inconsistent_values(df):
    for col in df:
        series = df[col]
        print(f" Series -->{col} unique values are {series.unique()} \n They are {series.nunique()} values \n")

In [29]:
check_inconsistent_values(md_df)

 Series -->State unique values are ['AL' 'FL' 'SD' 'MN' 'VA' 'OK' 'OH' 'MS' 'WI' 'IA' 'CA' 'IN' 'MO' 'MI'
 'NE' 'PA' 'AR' 'WV' 'KS' 'MA' 'KY' 'NY' 'VT' 'DC' 'IL' 'ND' 'SC' 'AK'
 'NM' 'NH' 'GA' 'NC' 'MD' 'TN' 'WA' 'TX' 'CO' 'NJ' 'LA' 'OR' 'AZ' 'ME'
 'ID' 'UT' 'RI' 'MT' 'PR' nan 'NV' 'CT' 'HI' 'WY'] 
 They are 51 values 

 Series -->Lat unique values are [34.3496  30.84513 43.54321 43.89744 37.59894 35.67302 41.67511 39.08062
 30.20097 45.40325 32.59914 28.66268 42.97168 42.03038 43.08426 34.79693
 33.74647 40.38836 38.40136 38.5404  44.96425 42.02415 41.13007 36.95315
 30.94714 41.70345 33.842   38.22553 41.45764 42.74733 37.03392 41.25883
 39.02887 35.83086 42.65331 43.80814 42.32731 42.38004 41.29905 40.94827
 38.15445 41.26466 38.88631 41.42553 33.22242 33.84778 39.33595 43.34063
 41.49075 38.41946 48.71825 39.27204 41.0012  36.59727 39.64778 36.58325
 42.25635 42.51238 30.23936 43.32154 46.04962 34.1589  33.46142 42.54215
 36.94006 63.67959 35.08098 43.07915 33.96739 40.10165 42.179

In [20]:

train_cat_data = X_train[cat_selector(X_train2)]
test_cat_data = X_test[cat_selector(X_train2)]
train_cat_data.head(10)


,State,Area,Marital,Gender,Initial_admin,Complication_risk,Services
82,TN,Urban,Never Married,Female,Emergency Admission,Medium,Intravenous
991,AL,Urban,Married,Male,Emergency Admission,Medium,Blood Work
789,TN,Urban,Married,Nonbinary,Observation Admission,High,Intravenous
894,SD,Rural,Never Married,Male,Observation Admission,High,Blood Work
398,MI,Suburban,Widowed,Female,Elective Admission,Medium,Blood Work
323,OH,Suburban,Separated,Female,Emergency Admission,Medium,Blood Work
519,OH,Urban,Separated,Female,Emergency Admission,High,Blood Work
916,CA,Suburban,Separated,Male,Emergency Admission,Low,Blood Work
922,TX,Urban,Divorced,Male,Emergency Admission,Medium,Intravenous
5,OK,Urban,Never Married,Male,Observation Admission,Medium,Blood Work


In [21]:
cat_selector(X_train2)

['State',
 'Area',
 'Marital',
 'Gender',
 'Initial_admin',
 'Complication_risk',
 'Services']

In [22]:
ohe_encode  = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_encode.fit(train_cat_data) 
train_ohe = ohe_encode.transform(train_cat_data)
test_ohe = ohe_encode.transform(test_cat_data) 
train_ohe  

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
ohe_column_names = ohe_encode.get_feature_names_out(train_cat_data.columns)
# redefining train and test ohe to a dataframe
train_ohe= pd.DataFrame(train_ohe, columns=ohe_column_names)
test_ohe = pd.DataFrame(test_ohe, columns=ohe_column_names)

train_ohe.head(10)

,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_FL,State_GA,...,Complication_risk_High,Complication_risk_Low,Complication_risk_Med,Complication_risk_Medium,Complication_risk_nan,Services_Blood Work,Services_CT Scan,Services_Intravenous,Services_MRI,Services_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [25]:
train_ohe

Index(['State_AK', 'State_AL', 'State_AR', 'State_AZ', 'State_CA', 'State_CO',
       'State_CT', 'State_DC', 'State_FL', 'State_GA', 'State_HI', 'State_IA',
       'State_ID', 'State_IL', 'State_IN', 'State_KS', 'State_KY', 'State_LA',
       'State_MA', 'State_MD', 'State_ME', 'State_MI', 'State_MN', 'State_MO',
       'State_MS', 'State_MT', 'State_NC', 'State_ND', 'State_NE', 'State_NH',
       'State_NJ', 'State_NM', 'State_NV', 'State_NY', 'State_OH', 'State_OK',
       'State_OR', 'State_PA', 'State_PR', 'State_RI', 'State_SC', 'State_SD',
       'State_TN', 'State_TX', 'State_UT', 'State_VA', 'State_VT', 'State_WA',
       'State_WI', 'State_WV', 'State_nan', 'Area_Rural', 'Area_Suburban',
       'Area_Urban', 'Area_nan', 'Marital_Divorced', 'Marital_Married',
       'Marital_Never Married', 'Marital_Separated', 'Marital_Widowed',
       'Marital_nan', 'Gender_F', 'Gender_Female', 'Gender_Male',
       'Gender_Nonbinary', 'Gender_f', 'Gender_m', 'Gender_male', 'Gender_nan',
   